In [2]:
import numpy as np
from manim import *

def generate_cone_points(n_points, seed=3, max_fraction=1.0):
    np.random.seed(seed)
    rs = np.sqrt(np.random.uniform(0, max_fraction**2, size=n_points))  # uniform in area
    thetas = np.random.uniform(0, 2 * np.pi, size=n_points)
    return list(zip(rs, thetas))
    
def point_on_cone(r_frac, theta, cone_length=10, cone_radius=4, radius_buffer_frac=0.9):
    # Move along the cone axis from tip (-6) to base (+4)
    x = -6 + r_frac * cone_length  # x ranges from -6 to +4

    # Maximum possible radius at this x position
    max_radius = r_frac * cone_radius

    # Shrink the actual radius to stay inside the cone
    radius = radius_buffer_frac * max_radius

    y = radius * np.cos(theta)
    z = radius * np.sin(theta)
    return np.array([x, y, z])
    

def point_on_circle(r_frac, theta, circle_radius=4):
    r = r_frac * circle_radius
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return np.array([x, y, 0])

def point_along_cone(fraction, height = 10, base_radius = 4):
        """Get a point along the axis of the cone and spread it out"""
        axis_point = LEFT * 3 + RIGHT * height * fraction
        radius = base_radius * fraction
        return axis_point + UP * (radius * 0.5) + OUT * (radius * 0.5)

def cone_point_to_circle(fraction, circle_radius = 4):
        r = circle_radius * fraction
        return np.array([r * 0.5, r * 0.5, 0])  # Approximate projection

## Redshift and LOS
Animation showing both side view of contamination (emission at different redshifts) and the line of sight view of contamination.

In [2]:
%%manim -qh -v WARNING combScene
from manim import *

class combScene(ThreeDScene):
    def construct(self):

        # Defines cone shape for animation
        base_radius = 4
        height = 10

        #Defines the number of 'LIM noise' around the base point
        num_points = 5

        #Initializes cones
        cone = Cone(
            base_radius=base_radius,
            height=height,
            direction=-X_AXIS,
            fill_opacity=0.15,
            fill_color=BLUE_B,
            stroke_width=0,
            resolution=(6, 16),
        ).shift(LEFT * 6)
        
        #Adds the initial cone so it starts on the screen
        self.add(cone)

        # Define cone that will 'grow' into (I know it is called initial_cone and so misleading, sorry!)
        initial_cone = Cone(
            base_radius=0.01,
            height=0.01,
            direction=-X_AXIS,
            fill_opacity=0.15,
            fill_color=BLUE_A,
            stroke_width=0,
            resolution=(6, 16),
        ).shift(LEFT * 6)

        
        #Defines Initial 3 emission dots
        dot1 = Dot3D(point_along_cone(1/5), color=BLUE_C)
        dot2 = Dot3D(point_along_cone(2/3), color=ORANGE)
        dot3 = Dot3D(point_along_cone(1/5)+[0, -0.75, -0.75], color=BLUE_C)

        
        #Defines reference dots (for label)
        ref_dot1 = Dot(color=BLUE_C).scale(0.7).to_corner(UL).shift(DOWN * 0.1 + RIGHT * 0.1)
        ref_label1 = Text("21cm Emission", font_size=20).next_to(ref_dot1, RIGHT, buff=0.2)
        ref_dot2 = Dot(color=ORANGE).scale(0.7).next_to(ref_dot1, DOWN, buff = 0.4)
        ref_label2 = Text("RRL Emission", font_size=20).next_to(ref_dot2, RIGHT, buff = 0.2)

        
        # Create a redshift label and arrow above the cone
        label = Text("Redshift", font_size=24).move_to(UP * 3.5)
        arrow = Arrow(
            start=LEFT * 3 + UP * 3,
            end=RIGHT * 3 + UP * 3,
            buff=0,
            stroke_width=4,
            color=RED
        )


        #Defines the 'LIM Noise' points
        thetas = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
        small_dots = VGroup()
        
        for theta in thetas:
            pos = dot1.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            pos2 = dot2.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            pos3 = dot3.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            small_dots.add(Dot3D(pos, radius=0.05, color=BLUE_B), Dot3D(pos2, radius=0.05, color=ORANGE), Dot3D(pos3, radius=0.05, color=BLUE_B))


        #Begin cone animation
        self.add(initial_cone)
        self.play(
            Transform(initial_cone, cone, replace_mobject_with_target_in_scene = False),
            GrowArrow(arrow), 
            FadeIn(label),
            run_time=5
        )
        
        self.wait(1)

        #Create initial emission dots and labels
        self.play(
            FadeIn(dot1),
            FadeIn(dot2),
            FadeIn(dot3),
            FadeIn(small_dots),
            FadeIn(VGroup(ref_dot1, ref_label1, ref_dot2, ref_label2)),
            run_time=2
        )


        #Creates vectorized mobject group for everything to do with the cone so I can move it easily
        coneGroup = VGroup(cone, initial_cone, dot1, dot2, dot3, small_dots, label, arrow)

        #Defines base circle on the cone
        base_circle = Circle(radius=base_radius, color=BLUE_B, fill_opacity=0.2)
        base_circle.rotate(angle=PI/2, axis=UP)  # Rotate from XY-plane to YZ-plane
        base_circle.move_to(cone.get_start())    # Move to the base of the cone
        base_circle.set_stroke(color=WHITE, width=2)

        #Defines where you want the final circle
        final_circle = base_circle.copy().rotate(-PI/2, axis=UP).move_to(RIGHT * 4).scale(0.6)

        #Creates circle on base of cone
        self.play(Create(base_circle), run_time = 1)
        
        self.wait(1)

        #Make base circle rotate and move to right side
        self.play(Transform(base_circle, final_circle, replace_mobject_with_target_in_scene = True), 
                  coneGroup.animate.scale(0.8).move_to(LEFT*3), run_time = 2)


        #Defines location of points in the new LOS circle from the location on the cone
        projected_dot1 = Dot(
            point_on_circle(1/5, np.pi/4, 4*0.6)+ base_circle.get_center(),
            color=BLUE_C
        )
        projected_dot2 = Dot(
            point_on_circle(2/3, np.pi/4, 4*0.6)+ base_circle.get_center(),
            color=ORANGE
        )

        projected_dot3 = Dot(
            point_on_circle(np.sqrt(2)/5, np.pi/4, 4*0.6)+ base_circle.get_center(),
            color=BLUE_C
        )


        #Defines the 'LIM Noise' points around the emission point for the LOS circle
        projected_small_dots = VGroup()
        for i in range(num_points):
            pos = projected_dot1.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            pos2 = projected_dot2.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            pos3 = projected_dot3.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            
            projected_small_dots.add(Dot(pos, radius=0.05, color=BLUE_B))
            projected_small_dots.add(Dot(pos2, radius=0.05, color=ORANGE))
            projected_small_dots.add(Dot(pos3, radius=0.05, color=BLUE_B))


        #Adds all the points in the new LOS circle
        self.play(
            FadeIn(projected_dot1),
            FadeIn(projected_dot2),
            FadeIn(projected_dot3),
            FadeIn(projected_small_dots),
            run_time=1
        )

        
        self.wait(3)


        #Adds all the newly 'observed' points to both the cone and LOS circle
        points_data = generate_cone_points(10)
        for i, point_pos in enumerate(points_data):
            #Generates cone point location and LOS point location
            cone_pt = point_on_cone(*point_pos, cone_length=10*0.6, cone_radius=4*0.6)
            proj_pt = base_circle.get_center() + point_on_circle(*point_pos, base_radius*0.6)
            surr_pts = VGroup()
            
            for j in range(num_points):
                #Generates the 'LIM Noise' points for each cone and LOS point
                fuzz_pt = proj_pt + [0.3*np.sin(thetas[j]), 0.3*np.cos(thetas[j]), 0]
                surr_pts.add(Dot(fuzz_pt, radius = 0.05, color = ORANGE))
                surr_pts.add(Dot3D(cone_pt+[0,0.3*np.cos(thetas[j]),0.3*np.sin(thetas[j])], radius=0.05, color=ORANGE))

            #Adds all the points to the scene
            self.add(Dot3D(point = cone_pt, color = ORANGE), Dot(point = proj_pt, color = ORANGE), surr_pts)
            self.wait(1)

        self.wait(5)
scene = combScene()
scene.render()

[06/03/25 10:51:09] INFO     Animation 0 : Using cached data (hash :                           ]8;id=793424;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=224851;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             4242032193_3330651110_658521764)                                                      

[06/03/25 10:51:11] INFO     Animation 1 : Partial movie file written in                   ]8;id=837527;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=581399;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/pipto/Documents/UW/Research/Animations/media/videos/10                         
                             80p60/partial_movie_files/combScene/2796882004_548731573_4044                         
                             57719.mp4'                                                                            

[06/03/25 10:51:13] INFO     Animation 2 : Using cached data (hash :                           ]8;id=222579;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=417629;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2796882004_2331656973_2099084129)                                                     

[06/03/25 10:51:17] INFO     Animation 3 : Partial movie file written in                   ]8;id=288362;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=784031;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/home/pipto/Documents/UW/Research/Animations/media/videos/10                         
                             80p60/partial_movie_files/combScene/2796882004_1242481674_637                         
                             785542.mp4'                                                                           

[06/03/25 10:51:19] INFO     Animation 4 : Using cached data (hash :                           ]8;id=619745;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=318490;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2796882004_2631580503_4149586690)                                                     

...

[06/03/25 10:52:46] INFO     Combining to Movie file.                                      ]8;id=720181;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=852674;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=860568;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=65284;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/pipto/Documents/UW/Research/Animations/media/videos/10                         
                             80p60/combScene.mp4'                                                                  
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=475940;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=819661;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#856\856]8;;\
                             manim has removed the 3 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered combScene                                                        ]8;id=167214;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=597016;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 19 animations                                                                  

Manim Community v0.19.0

## Independent Cone (redshift) Animation

In [3]:
%%manim -qh -v WARNING LCScene
from manim import *

class LCScene(ThreeDScene):
    def construct(self):

        #Defines cone shape for animation
        base_radius = 4
        height = 10

        #Defines the number of 'LIM noise' around the base point
        num_points = 5

        #Defines number of observations to add
        n_obs = 10

        #Initializes cones
        cone = Cone(
            base_radius=base_radius,
            height=height,
            direction=-X_AXIS,
            fill_opacity=0.15,
            fill_color=BLUE_B,
            stroke_width=0,
            resolution=(6, 16), 
        ).shift(LEFT * 6)

        #Adds the initial cone so it starts on the screen
        self.add(cone)

        
        #Define cone that will 'grow' into (I know it is called initial_cone and so misleading, sorry!)
        initial_cone = Cone(
            base_radius=0.01,
            height=0.01,
            direction=-X_AXIS,
            fill_opacity=0.15,
            fill_color=BLUE_A,
            stroke_width=0,
            resolution=(6, 16),
        ).shift(LEFT * 6)

        
        #Defines Initial 3 emission dots
        dot1 = Dot3D(point_along_cone(1/5), color=BLUE_C)
        dot2 = Dot3D(point_along_cone(2/3), color=ORANGE)
        dot3 = Dot3D(point_along_cone(1/5)+[0, -0.75, -0.75], color=BLUE_C)

        
        dot1_pos = dot1.get_center()
        dot3_pos = dot3.get_center()

        #Defines reference dots (for label)
        ref_dot1 = Dot(color=BLUE_C).scale(0.7).to_corner(UL).shift(DOWN * 0.1 + RIGHT * 0.1)
        ref_label1 = Text("21cm Emission", font_size=20).next_to(ref_dot1, RIGHT, buff=0.2)
        
        ref_dot2 = Dot(color=ORANGE).scale(0.7).next_to(ref_dot1, DOWN, buff = 0.4)
        ref_label2 = Text("RRL Emission", font_size=20).next_to(ref_dot2, RIGHT, buff = 0.2)

        
        # Create a redshift label and arrow above the cone
        label = Text("Redshift", font_size=24).move_to(UP * 3.5)
        arrow = Arrow(
            start=LEFT * 3 + UP * 3,
            end=RIGHT * 3 + UP * 3,
            buff=0,
            stroke_width=4,
            color=RED
        )


        #Defines the 'LIM Noise' points
        thetas = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
        small_dots = VGroup()
        
        for theta in thetas:
            pos = dot1.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            pos2 = dot2.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            pos3 = dot3.get_center() + [0,0.3*np.cos(theta),0.3*np.sin(theta)]
            small_dots.add(Dot3D(pos, radius=0.05, color=BLUE_B), Dot3D(pos2, radius=0.05, color=ORANGE), Dot3D(pos3, radius=0.05, color=BLUE_B))


        #Begin cone animation
        self.add(initial_cone)
        self.play(
            Transform(initial_cone, cone),
            GrowArrow(arrow), 
            FadeIn(label),
            run_time=5
        )
        
        self.wait(1)

        #Create initial emission dots and labels
        self.play(
            FadeIn(dot1),
            FadeIn(dot2),
            FadeIn(dot3),
            FadeIn(small_dots),
            FadeIn(VGroup(ref_dot1, ref_label1, ref_dot2, ref_label2)),
            run_time=2
        )
        
        self.wait(5)

        #Adds all the newly 'observed' points the cone
        close_enough = 1.65
        points_data = generate_cone_points(n_obs)
        for i, point_pos in enumerate(points_data):
            #Generates more observation points on the cone
            cone_pt = point_on_cone(*point_pos, cone_length=10, cone_radius=4)
            surr_pts = VGroup()
            
            for j in range(num_points):
                #Generates the 'LIM Noise' around each cone point, and checks if they are 'close enough' to contaminate physical space
                surr_pts.add(Dot3D(cone_pt+[0,0.3*np.cos(thetas[j]),0.3*np.sin(thetas[j])], radius=0.05, color=GREEN_C if ((np.linalg.norm(cone_pt - dot1_pos) <= close_enough) or (np.linalg.norm(cone_pt - dot3_pos) <= close_enough)) else ORANGE))
            
            #Adds all the points to the scene
            self.add(Dot3D(point = cone_pt, color=GREEN_C if ((np.linalg.norm(cone_pt - dot1_pos) <= close_enough) or (np.linalg.norm(cone_pt - dot3_pos) <= close_enough)) else ORANGE), surr_pts)
            self.wait(1)

        self.wait(5)
scene = LCScene()
scene.render()

Manim Community v0.19.0

## Independent LOS View

In [3]:
%%manim -v WARNING CircleViewScene

from manim import *
import numpy as np


#Makes the animation output a square in the config file
config = {
    "frame_width": 10.0,
    "frame_height": 10.0,
    "pixel_width": 1920,
    "pixel_height": 1920,
}


class CircleViewScene(Scene):
    def construct(self):

        #Defines cone shape for animation (I have it as 3.9 not 4 because 4 gets cut off in this case!
        circle_radius = 3.9

        #Defines the number of 'LIM noise' around the base point
        num_points = 5

        #Defines number of observations to add
        n_obs = 10

        
        #Defines circle shape
        circle = Circle(radius=circle_radius, color=BLUE_B, fill_opacity=0.2)

        #Creates the base circle
        self.play(DrawBorderThenFill(circle), run_time = 4)


        #Defines Initial 3 emission dots
        dot1_pos = cone_point_to_circle(1/5, circle_radius = circle_radius)
        dot2_pos = cone_point_to_circle(2/3, circle_radius = circle_radius)
        dot3_pos = dot1_pos + [-0.3, -0.3, 0]

        dot1 = Dot(dot1_pos, color=BLUE_C)
        dot2 = Dot(dot2_pos, color=ORANGE)
        dot3 = Dot(dot3_pos, color=BLUE_C)


        #Defines the 'LIM Noise' points
        thetas = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
        projected_small_dots = VGroup()
        for i in range(num_points):
            pos = dot1.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            pos2 = dot2.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            pos3 = dot3.get_center() + [0.3*np.sin(thetas[i]), 0.3*np.cos(thetas[i]), 0]
            
            projected_small_dots.add(Dot(pos, radius=0.05, color=BLUE_B))
            projected_small_dots.add(Dot(pos2, radius=0.05, color=ORANGE))
            projected_small_dots.add(Dot(pos3, radius=0.05, color=BLUE_B))

        
        #Defines reference dots (for label)
        ref_dot1 = Dot(color=BLUE_C).scale(0.7).to_corner(UL).shift(DOWN * 0.1 + RIGHT * 0.1)
        ref_label1 = Text("21cm Emission", font_size=20).next_to(ref_dot1, RIGHT, buff=0.2)
        ref_dot2 = Dot(color=ORANGE).scale(0.7).next_to(ref_dot1, DOWN, buff=0.4)
        ref_label2 = Text("RRL Emission", font_size=20).next_to(ref_dot2, RIGHT, buff=0.2)

        
        self.wait(2)

        #Create initial emission dots and labels
        self.play(FadeIn(dot1), 
                  FadeIn(dot2), 
                  FadeIn(dot3), 
                  FadeIn(projected_small_dots), 
                  FadeIn(VGroup(ref_dot1, ref_label1, ref_dot2, ref_label2)),
                  run_time=2
                 )

        
        self.wait(5)

        #Adds all the newly 'observed' points the LOS circle
        points_data = generate_cone_points(n_obs)
        for i, point_pos in enumerate(points_data):
            #Generates observation points on the circle
            proj_pt = circle.get_center() + point_on_circle(*point_pos)
            surr_pts = VGroup()
            
            for j in range(num_points):
                #Generates the 'LIM Noise' points for each cone and LOS point
                fuzz_pt = proj_pt + [0.3*np.sin(thetas[j]), 0.3*np.cos(thetas[j]), 0]
                surr_pts.add(Dot(fuzz_pt, radius = 0.05, color = GREEN if (i == 4 or i == 9) else ORANGE))
            
            #Adds all the points to the scene
            self.add(Dot(point = proj_pt, color = GREEN if (i == 4 or i == 9) else ORANGE), surr_pts)
            self.wait(1)

        self.wait(5)
scene = CircleViewScene()
scene.render()

[06/03/25 11:20:06] INFO     Animation 0 : Using cached data (hash :                           ]8;id=694016;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=153299;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             3977891868_1676021164_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=271840;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=270978;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2852726489_2844360058_1943612289)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=381245;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=439849;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2852726489_1359013670_797476840)                                                      

[06/03/25 11:20:07] INFO     Animation 3 : Using cached data (hash :                           ]8;id=245233;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=51170;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2852726489_1038707267_2036788994)                                                     

                    INFO     Animation 4 : Using cached data (hash :                           ]8;id=731895;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=791697;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2852726489_4217992463_1543717424)                                                     

...

                    INFO     Animation 14 : Using cached data (hash :                          ]8;id=194941;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=851631;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/renderer/cairo_renderer.py#89\89]8;;\
                             2852726489_1038707267_1127054272)                                                     

                    INFO     Combining to Movie file.                                      ]8;id=61770;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=217373;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=716233;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=553032;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/home/pipto/Documents/UW/Research/Animations/media/videos/10                         
                             80p60/CircleViewScene.mp4'                                                            
                                                                                                                   

                    INFO     Rendered CircleViewScene                                                  ]8;id=436264;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=535320;file:///home/pipto/manim-env/lib/python3.11/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 15 animations                                                                  

Manim Community v0.19.0

## Power Spectrum Animation

In [4]:
%%manim -qh -v WARNING JumpingParabolas
from manim import *

class JumpingParabolas(Scene):
    def construct(self):

        n_obs = 10
        #Defines the coordinate system
        axes = Axes(
            x_range=[0, 10],
            y_range=[0, 6],
            x_length=10,
            y_length=6,
        )

        #Defines labels of coordinate system
        labels = axes.get_axis_labels(
            Text("").scale(0.45), Text("Power Spectrum").scale(0.45)
        )

        #Adds coordsystem and label to scene
        self.add(axes, labels)
        
        self.wait(6)

        #Generates 21cm curve
        def rrl_curve(x):
            return -(x-5)**2 / 15 + 5
        base_curve = axes.plot(rrl_curve, color=BLUE)
        

        # Label for both curves
        label_21cm = Text("21cm Auto PS", color=BLUE).scale(0.5).to_corner(UR).shift(DOWN * 0.5)
        label_rrl = Text("RRL Auto PS", color=ORANGE).scale(0.5).next_to(label_21cm, DOWN, aligned_edge=RIGHT)
        label_rrlCross = Text("RRL-21 Cross PS (Effect)", color=GREEN).scale(0.5).next_to(label_rrl, DOWN, aligned_edge=RIGHT)

        
        # Animate RRL curve jumping up with observation
        for i in range(1, (n_obs+2)):
            #How much line should jump by
            alpha = 1.6 + i * 0.1

            # Line to jump
            def updated_curve(x, a=alpha):
                return -(x-5)**2 / 15 + a
            new_curve = axes.plot(lambda x: updated_curve(x), color=ORANGE)


            if i == 1:
                #As a first step, add in the RRL curve, the 21cm curve, and all the labels
                self.play(FadeIn(new_curve), FadeIn(base_curve), FadeIn(label_21cm), FadeIn(label_rrl), FadeIn(label_rrlCross), run_time = 2)
                self.wait(4)
            else:
                self.add(new_curve)
                
            if i == 6:
                #Hard coded version of which point is 'close enough' from previous animation to contaminate
                self.add(axes.plot(lambda x: (-190*alpha*(x-2.45)*(x-2.68)*(x-2.85)*(x-3.3)*(x-3.7)*(x-3.4)) if x > 2.2 and x < 3.6 else -10, color=GREEN))
                
            if i == 11:
                #Hard coded version of which point is 'close enough' from previous animation to contaminate
                self.add(axes.plot(lambda x: (-220*alpha*(x-2.7)*(x-2.93)*(x-3.1)*(x-3.55)*(x-3.85)*(x-4)) if x > 2.45 and x < 4.2 else -10, color=GREEN))
                
            self.wait(1)
            
            if i < (n_obs+1):
                self.remove(new_curve)
                
        self.wait(5)
scene = JumpingParabolas()
scene.render()

Manim Community v0.19.0